In [21]:
import librosa
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import torchmetrics
import os
import warnings

warnings.filterwarnings('ignore')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class Config:
    SR = 32000
    N_MFCC = 100
    # Dataset
    ROOT_FOLDER = './'
    # Training
    N_CLASSES = 2
    BATCH_SIZE = 96
    N_EPOCHS = 50
    LR = 3e-4
    # Others
    SEED = 42
    
CONFIG = Config()

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED) # Seed 고정

df = pd.read_csv('./train.csv')
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CONFIG.SEED)

def get_mfcc_feature(df, train_mode=True):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows()):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(row['path'], sr=CONFIG.SR)
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        features.append(mfcc)

        if train_mode:
            label = row['label']
            label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)

    if train_mode:
        return features, labels
    return features



class CustomDataset(Dataset):
    def __init__(self, mfcc, label):
        self.mfcc = mfcc
        self.label = label

    def __len__(self):
        return len(self.mfcc)

    def __getitem__(self, index):
        if self.label is not None:
            return self.mfcc[index], self.label[index]
        return self.mfcc[index]



In [3]:
train_mfcc, train_labels = get_mfcc_feature(train, True)
val_mfcc, val_labels = get_mfcc_feature(val, True)

44350it [07:06, 103.98it/s]
11088it [01:47, 103.49it/s]


In [4]:
np.save("./train_Mfcc.npy", train_mfcc)
np.save("./train_Label.npy", train_labels)
np.save("./val_Mfcc.npy", val_mfcc)
np.save("./val_Label.npy", val_labels)

In [22]:
train_mfcc = np.load("./train_Mfcc.npy")
train_labels = np.load("./train_Label.npy")
val_mfcc = np.load("./val_Mfcc.npy")
val_labels = np.load("./val_Label.npy")

In [23]:
train_dataset = CustomDataset(train_mfcc, train_labels)
val_dataset = CustomDataset(val_mfcc, val_labels)
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

In [24]:
from sklearn.metrics import roc_auc_score

def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CONFIG.N_EPOCHS+1):
        model.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(features)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}]')
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score
    
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            probs = model(features)
            
            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        
        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)
        
        # Calculate AUC score
        auc_score = multiLabel_AUC(all_labels, all_probs)
    
    return _val_loss, auc_score

class MLP(nn.Module):
    def __init__(self, input_dim=CONFIG.N_MFCC, hidden_dim=128, output_dim=CONFIG.N_CLASSES):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return torch.sigmoid(x)

# Autoencoder for pretraining
class Autoencoder(nn.Module):
    def __init__(self, input_dim=CONFIG.N_MFCC, hidden_dim=128):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

def get_mfcc_feature_unlabeled(file_paths):
    features = []
    for file_path in tqdm(file_paths):
        y, sr = librosa.load(file_path, sr=CONFIG.SR)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        features.append(mfcc)
    return features



def pretrain_autoencoder(autoencoder, optimizer, dataloader, device):
    autoencoder.train()
    for epoch in range(CONFIG.N_EPOCHS):
        train_loss = []
        for features in tqdm(dataloader):
            features = features.float().to(device)
            
            optimizer.zero_grad()
            
            reconstructed = autoencoder(features)
            loss = criterion(reconstructed, features)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        print(f'Epoch [{epoch+1}/{CONFIG.N_EPOCHS}], Loss: {np.mean(train_loss):.5f}')
    
    return autoencoder



class PretrainedMLP(nn.Module):
    def __init__(self, autoencoder, input_dim=CONFIG.N_MFCC, hidden_dim=128, output_dim=CONFIG.N_CLASSES):
        super(PretrainedMLP, self).__init__()
        self.encoder = autoencoder.encoder
        self.fc1 = nn.Linear(hidden_dim // 2, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.encoder(x)
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return torch.sigmoid(x)

In [25]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)
            
            probs = model(features)
            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

In [9]:
unlabeled_file_paths = [os.path.join(CONFIG.ROOT_FOLDER, './unlabeled_data', f) for f in os.listdir(os.path.join(CONFIG.ROOT_FOLDER, './unlabeled_data'))]
unlabeled_mfcc = get_mfcc_feature_unlabeled(unlabeled_file_paths)


100%|██████████| 1264/1264 [00:15<00:00, 80.62it/s]


In [10]:
np.save("./unlabeled_Mfcc.npy", unlabeled_mfcc)

In [26]:
unlabeled_mfcc = np.load("./unlabeled_Mfcc.npy")

In [27]:

unlabeled_dataset = CustomDataset(unlabeled_mfcc, None)
unlabeled_loader = DataLoader(
    unlabeled_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=True
)

autoencoder = Autoencoder().to(device)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=CONFIG.LR)
criterion = nn.MSELoss()

In [28]:
autoencoder = pretrain_autoencoder(autoencoder, optimizer, unlabeled_loader, device)

100%|██████████| 14/14 [00:00<00:00, 571.30it/s]


Epoch [1/50], Loss: 854.60808


100%|██████████| 14/14 [00:00<00:00, 608.71it/s]


Epoch [2/50], Loss: 839.15180


100%|██████████| 14/14 [00:00<00:00, 560.01it/s]


...

Epoch [48/50], Loss: 841.37097


100%|██████████| 14/14 [00:00<00:00, 777.77it/s]

Epoch [49/50], Loss: 830.58067



100%|██████████| 14/14 [00:00<00:00, 823.55it/s]

Epoch [50/50], Loss: 836.95370


In [29]:
pretrained_mlp = PretrainedMLP(autoencoder).to(device)
optimizer = torch.optim.Adam(pretrained_mlp.parameters(), lr=CONFIG.LR)
infer_model = train(pretrained_mlp, optimizer, train_loader, val_loader, device)
torch.save(infer_model.state_dict(), "./pretrained_mlp_model_03.pth")

100%|██████████| 116/116 [00:00<00:00, 829.91it/s]


Epoch [1], Train Loss : [0.23914] Val Loss : [0.04766] Val AUC : [0.99867]


100%|██████████| 116/116 [00:00<00:00, 778.39it/s]


Epoch [2], Train Loss : [0.04669] Val Loss : [0.03550] Val AUC : [0.99913]


100%|██████████| 116/116 [00:00<00:00, 811.10it/s]


...

Epoch [48], Train Loss : [0.00068] Val Loss : [0.00577] Val AUC : [0.99998]


100%|██████████| 116/116 [00:00<00:00, 720.51it/s]


Epoch [49], Train Loss : [0.00197] Val Loss : [0.00822] Val AUC : [0.99996]


100%|██████████| 116/116 [00:00<00:00, 791.75it/s]

Epoch [50], Train Loss : [0.00178] Val Loss : [0.00677] Val AUC : [0.99997]


In [15]:
val2 = pd.read_csv('./test.csv')
val2_mfcc = get_mfcc_feature(val2, False)
val2_dataset = CustomDataset(val2_mfcc, None)
val2_loader = DataLoader(
    val2_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

50000it [10:07, 82.27it/s]


In [16]:
np.save("./test_Mfcc.npy", val2_mfcc)

In [39]:
val2_mfcc = np.load("./train_Mfcc.npy")
val3_mfcc = np.load("./val_Mfcc.npy")
val2_mfcc = np.concatenate((val2_mfcc, val3_mfcc), axis=0)
val2_dataset = CustomDataset(val2_mfcc, None)
val2_loader = DataLoader(
    val2_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

In [40]:
preds = inference(infer_model, val2_loader, device)

100%|██████████| 578/578 [00:00<00:00, 1627.76it/s]


In [41]:
submit = pd.read_csv('./train_data.csv')
submit.iloc[:, 1:] = preds
submit.head()

,id,fake,real
0,RUNQPNJF,1.0,0.0
1,JFAWUOGJ,0.999878,0.000124
2,RDKEKEVX,0.0,1.0
3,QYHJDOFK,0.999999,0.000001
4,RSPQNHAO,0.0,1.0


In [36]:
submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = preds
submit.head()

ValueError: Length of values (44350) does not match length of index (50000)

In [42]:
submit.to_csv('train_pre.csv',index=False)